In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.pipeline import Pipeline
from  Auto_encoders import AutoEncoder
from preprocessing import DataPreprocessor
from Predictors import CorrelationAnalysis
from Model_Training_boosting import ENSOAnalysis_br

In [ ]:
class execute:
    
    def __init__(self, nc_file1, nc_file2, enso_file):
        self.nc_file1 = nc_file1
        self.nc_file2 = nc_file2
        self.enso_file = enso_file
    
    def ploting_correlation(self):
        autoencoder = AutoEncoder()
        processor = DataPreprocessor()
        autoencoder_input_data = processor.execute_pipeline(self.nc_file1,self.nc_file2)
        
        autoencoder.fit_autoencoder(autoencoder_input_data)
        weights = autoencoder.weights
        input_data = autoencoder.input_data
        correlation_analyzer = CorrelationAnalysis(input_data, weights, self.enso_file )
        top_predictions, sorted_correlation_data = correlation_analyzer.run_correlation_analysis()
        pearson = top_predictions['top_pred_pearson']
        kendall = top_predictions['top_pred_kendall']
        spearman = top_predictions['top_pred_spearman']
        mutual = top_predictions['top_pred_mutual']
        correlations = [pearson, kendall, spearman, mutual]
        analysis = ENSOAnalysis_br(correlations, self.enso_file)
        analysis.execute_analysis(['mam_Avg','Mar', 'Apr','May'])
        return analysis

In [ ]:
nc_file1 = "AIRT2023.nc"
nc_file2 = "GPH200.nc"

enso_file = "enso2023.csv"

# Create an instance of the execute class
executor = execute(nc_file1, nc_file2, enso_file)

# Call the ploting_correlation method
analysis = executor.ploting_correlation()

In [ ]:
with open('best_models_boosting.pkl', 'wb') as f:
    pickle.dump(analysis.best_models, f)

In [ ]:
with open('best_models.pkl', 'rb') as f:
    best_models = pickle.load(f)

In [ ]:
with open('best_scalers_boosting.pkl', 'wb') as f:
    pickle.dump(analysis.scalers, f)

In [ ]:
with open('best_scalers.pkl', 'rb') as f:
    best_scalers = pickle.load(f)

In [ ]:
with open('y_preds.pkl', 'rb') as f:
   best_preds = pickle.load(f)

In [ ]:
with open('y_tests.pkl', 'wb') as f:
    pickle.dump(analysis.y_tests, f)

In [ ]:
with open('y_tests.pkl', 'rb') as f:
   best_tests = pickle.load(f)

In [ ]:
best_tests_mar = best_tests['Mar']
best_tests_mar

In [ ]:
import numpy as np

def convert_to_categorical(data):
  """
  Converts numerical data to categorical (0 for < 0, 1 otherwise).

  Args:
      data (np.ndarray): The data to be converted.

  Returns:
      np.ndarray: The converted categorical data.
  """
  return np.where(data < 0, 0, 1)

# Apply the conversion to y_pred and y_test
y_pred_categorical = convert_to_categorical(best_preds_mar)
y_test_categorical = convert_to_categorical(best_tests_mar)

In [ ]:
y_pred = y_pred_categorical
y_test = y_test_categorical

In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix

# Example data (replace this with your actual y_test and y_pred)
# Calculate metrics

recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
# Calculate Sensitivity
TP = conf_matrix[1, 1]
FN = conf_matrix[1, 0]
sensitivity = TP / (TP + FN)

# Calculate Specificity
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
specificity = TN / (TN + FP)

# Calculate Negative Predictive Rate
negative_predictive_rate = TN / (TN + FN)

# Create a DataFrame to store the metrics
data = {
    'Metric': ['Correlation','Sensitivity','Specificity','Precision','Negative Predictive Rate','Accuracy','F1 Score'],
    'MAR': [.94,sensitivity,specificity,precision, negative_predictive_rate,accuracy, f1]
}
metrics_df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
metrics_df.to_csv('model_metrics_Mar.csv', index=False)
